In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


#Fake data creation

In [2]:
from numpy import integer


fake=Faker()

fake_data = defaultdict(list)

row_number=int(input("Please write the number you wish to see the row size, DON'T CRASH ME!"))



In [3]:

for i in range(row_number):
    fake_data['customer1'].append(fake.bothify(text='16######'))
    fake_data['amount'].append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
    fake_data['timestamp'].append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
    

In [4]:
copy_list=fake_data['customer1'].copy()
random.shuffle(copy_list)
fake_data['customer2']=copy_list
fake_data

defaultdict(list,
            {'customer1': ['16730364', '16561597', '16130704'],
             'amount': [3570.22, 2465.21, 1544.33],
             'timestamp': [datetime.datetime(2022, 9, 29, 5, 40, 30),
              datetime.datetime(2022, 10, 15, 4, 53, 20),
              datetime.datetime(2022, 9, 21, 6, 57, 46)],
             'customer2': ['16130704', '16561597', '16730364']})

#Adding dict to dataframe

In [5]:
df=pd.DataFrame(fake_data)

df=df[['customer1','customer2','amount','timestamp']]

print(df)

  customer1 customer2   amount           timestamp
0  16730364  16130704  3570.22 2022-09-29 05:40:30
1  16561597  16561597  2465.21 2022-10-15 04:53:20
2  16130704  16730364  1544.33 2022-09-21 06:57:46


#Same value change with random

In [24]:
liste=[]
a=df['customer1'].tolist()
for i in range(len(a)):
    b=random.choice(a)
    if b==a[i]:
        continue
    liste.append(random.choice(a))

liste

['16730364', '16130704']

In [20]:
same_value_list=[]
for i in range(len(df)):
    while df.iloc[i,0] == df.iloc[i,1]:
        pass
        
print(same_value_list)

df

KeyboardInterrupt: 